This script is used to find out how model performance changes with number of predictors for the full-predictor, first-term model.

In [1]:
import pickle
import pandas as pd
import numpy as np
from collections import Counter
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve, roc_auc_score, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import KFold, StratifiedKFold
from scipy.stats.mstats import gmean
import seaborn as sns
import matplotlib.pyplot as plt

results_dir = "~\\Box Sync\\Clickstream\\evaluation_results\\first\\"

In [2]:
sn_dict = {"Blue Ridge": "BRCC",
           "Central Virginia": "CVCC",
           "Dabney S. Lancaster": "DSLCC",
           "Danville": "DCC",
           "Eastern Shore": "ESCC",
           "Germanna": "GCC",
           'J. Sargeant Reynolds': "JSRCC",
           'John Tyler': "JTCC",
           "Lord Fairfax": "LFCC",
           "Mountain Empire": "MECC",
           "New River": "NRCC",
           "Northern Virginia": "NVCC",
           "Patrick Henry": "PHCC",
           "Paul D. Camp": "PDCCC",
           "Piedmont Virginia": "PVCC",
           "Rappahannock": "RCC",
           "Southside Virginia": "SSVCC",
           "Southwest Virginia": "SWVCC",
           "Thomas Nelson": "TNCC",
           "Tidewater": "TCC",
           "Virginia Highlands": "VHCC",
           "Virginia Western": "VWCC",
           "Wytheville": "WCC"}

In [3]:
df0 = pd.read_stata("~\\Box Sync\\Clickstream\\data\\first\\LMS_data_final.dta")
df1 = pd.read_csv("~\\Box Sync\\Clickstream\\data\\first\\course_specific_predictors_new.csv")
df2 = pd.read_csv("~\\Box Sync\\Clickstream\\data\\first\\term_specific_predictors_new.csv")
for v in [int(e) for e in np.unique(df2.cip) if e != 0]:
    df2.loc[:,'cip_'+str(v)] = (df2.cip == v).astype(int)
for v in [int(e) for e in np.unique(df2.degree_level) if e != 4]:
    df2.loc[:,'degree_level_'+str(v)] = (df2.degree_level == v).astype(int)
df2 = df2.drop(['cip', 'degree_level'], axis=1)
df4 = pd.read_stata("~\\Box Sync\\Clickstream\\data\\first\\instructor_related_predictors.dta")
df5 = df0.loc[:,['vccsid','strm','college','course','section']].copy()
df5.loc[:,'college_new'] = df5.college.apply(lambda x: sn_dict[x])
for sn in [e for e in sn_dict.values() if e != "BRCC"]:
    df5.loc[:,'college_'+sn] = (df5.college_new == sn).astype(int)
df5 = df5.drop(['college_new'], axis=1)
df = df0.merge(df1, how='inner', on=['vccsid','strm','college','course','section'])\
.merge(df2, how='inner', on=['vccsid','strm'])\
.merge(df4, how='inner', on=['vccsid','strm','college','course','section'])\
.merge(df5, how='inner', on=['vccsid','strm','college','course','section'])
predictors = [e for e in list(df.columns)[5:] if e != "grade"]
len(predictors)

80

In [4]:
assert pd.isnull(df).any().any() == False

In [5]:
df.shape

(204853, 86)

In [6]:
train_df = df[df.strm != 2212]
test_df = df[df.strm == 2212]
original_test_grade = np.array(test_df.grade)
train_df.loc[:,'grade'] = train_df.apply(lambda x: 1 if x.loc['grade'] in {'A','B','C'} else 0, axis=1)
test_df.loc[:,'grade'] = test_df.apply(lambda x: 1 if x.loc['grade'] in {'A','B','C'} else 0, axis=1)
print(train_df.shape,test_df.shape)

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(181673, 86) (23180, 86)


In [9]:
# raw_predictors = {\
# 10: 'tot_click_cnt_qrt1, crnt_enrl_intensity, tot_time_qrt1, disc_reply_cnt_qtr1, disc_post_cnt_qtr1, avg_g, assign_sub_cnt_qtr1, has_assign_sub_cnt_qtr1, past_avg_grade, past_avg_grade',
# 20: 'avg_depth_post_qtr1, on_time_assign_share_qtr1, has_on_time_assign_share_qtr1, has_concurrent_qtr1, tot_click_cnt_qrt1c, avg_word_tot_qtr1, tot_time_qrt1c, irreg_session_len_qrt1, summer_ind, avg_session_len_qrt1',
# 30: 'irreg_session_len_qrt1c, has_avg_g_concurrent, avg_g_concurrent, age, avg_session_len_qrt1c, section_size, assign_sub_cnt_qtr1c, has_assign_sub_cnt_qtr1c, lvl2_share, online_share',
# 40: 'has_on_time_assign_share_qtr1c, on_time_assign_share_qtr1c, dev, eve_share, degree_level_1, online_ind, full_time, tenure, college_NVCC, degree_level_2',
# 50: 'lvl2_ind, college_TCC, cip_24, cip_52, degree_level_3, college_JTCC, cip_51, cip_11, eve_ind, cip_30',
# 60: 'college_JSRCC, college_GCC, college_TNCC, cip_45, college_LFCC, college_VWCC, cip_43, college_SWVCC, cip_14, cip_50',
# 70: 'cip_15, cip_99, college_CVCC, college_NRCC, college_PVCC, cip_47, college_PHCC, cip_19, college_VHCC, cip_42',
# 80: 'college_MECC, college_DCC, college_RCC, college_WCC, college_PDCCC, college_SSVCC, cip_48, college_DSLCC, cip_46, college_ESCC'}

In [16]:
raw_predictors = {\
5: 'tot_click_cnt_qrt1, crnt_enrl_intensity, tot_time_qrt1, disc_reply_cnt_qtr1, disc_post_cnt_qtr1',
15: 'avg_g, assign_sub_cnt_qtr1, has_assign_sub_cnt_qtr1, past_avg_grade, has_past_avg_grade, avg_depth_post_qtr1, on_time_assign_share_qtr1, has_on_time_assign_share_qtr1, has_concurrent_qtr1, tot_click_cnt_qrt1c',
25: 'avg_word_tot_qtr1, tot_time_qrt1c, irreg_session_len_qrt1, summer_ind, avg_session_len_qrt1, avg_g_concurrent, has_avg_g_concurrent, irreg_session_len_qrt1c, age, avg_session_len_qrt1c',
35: 'section_size, assign_sub_cnt_qtr1c, has_assign_sub_cnt_qtr1c, lvl2_share, on_time_assign_share_qtr1c, has_on_time_assign_share_qtr1c, online_share, dev, eve_share, degree_level_1',
45: 'online_ind, full_time, tenure, college_NVCC, degree_level_2, lvl2_ind, college_TCC, cip_24, cip_52, degree_level_3'}

In [17]:
clean_predictors = {}
for k,v in raw_predictors.items():
    clean_predictors[k] = [p.strip() for p in v.split(",")]
num_of_predictors = sorted(list(clean_predictors.keys()))
clean_predictors_2 = {}
for k,v in clean_predictors.items():
    clean_predictors_2[k] = []
    for n in num_of_predictors:
        if n <= k:
            clean_predictors_2[k] += clean_predictors[n]
    assert len(clean_predictors_2[k]) == k

In [18]:
def create_cv_folds(train, predictors, n_fold = 5):
    folds = []
    k_fold = StratifiedKFold(n_splits = n_fold, random_state = 12345, shuffle=True)
    for train_indices, test_indices in k_fold.split(train, train.grade):
        train_part = train.iloc[train_indices,:]
        test_part = train.iloc[test_indices,:]
        X_1 = train_part.loc[:,predictors]
        y_1 = train_part.grade
        X_2 = test_part.loc[:,predictors]
        y_2 = test_part.grade
        folds.append([(X_1.copy(),y_1.copy()),(X_2.copy(),y_2.copy())])
    return folds

In [19]:
def cross_validation_RF(rf_model, folds):
    auc_by_fold = []
    for f in folds:
        X_1 = f[0][0]
        y_1 = f[0][1]
        X_2 = f[1][0]
        y_2 = f[1][1]
        rf_model.fit(X_1,y_1)
        y_2_pred = rf_model.predict_proba(X_2)[:,1]
        auc_by_fold.append(roc_auc_score(y_2,y_2_pred))
    return round(np.mean(auc_by_fold),4)  

In [20]:
def calc_cw(y):
    # Calculate the weight of each letter grade to be used in the modeling fitting procedure: the weight is inversely proportional to the square root of the frequency of the letter grade in the training sample
    cw = Counter(y)
    class_weight = {k:np.sqrt(cw.most_common()[0][-1]/v, dtype=np.float32) for k,v in cw.items()}
    return class_weight # The output is a dictionary mapping letter grade to the corresponding weight

In [21]:
cstat_by_num = []
for n in sorted(list(clean_predictors_2.keys())):
    print("n = {}:\n".format(n))
    predictors_list = clean_predictors_2[n]
    five_folds = create_cv_folds(train_df, predictors_list)
    auc_by_d=[]
    for d in range(5,36):
        rf = RandomForestClassifier(n_estimators=200, criterion="entropy", 
                                    max_depth=d,
                                    random_state=0, n_jobs=20, max_features="auto",
                                    class_weight = calc_cw(train_df.grade))
        auc = cross_validation_RF(rf, five_folds)
        auc_by_d.append(auc)
        print("max_depth = {0}: {1}".format(d, auc))
        if d > 5:
            if auc - auc_by_d[-2] < 0.001:
                break
            else:
                best_d = d
        else:
            best_d = d
    rf = RandomForestClassifier(n_estimators=200, criterion="entropy",
                                max_depth=best_d,
                                random_state=0, n_jobs=-1, max_features="auto",
                                class_weight = calc_cw(train_df.grade))
    rf.fit(train_df.loc[:,predictors_list], train_df.grade)
    cstat_by_num.append(round(roc_auc_score(test_df.grade, rf.predict_proba(test_df.loc[:,predictors_list])[:,1]),4))
    print("\nC-statistic = {}".format(cstat_by_num[-1]))
    print("\n\n\n")

n = 5:

max_depth = 5: 0.7225
max_depth = 6: 0.7264
max_depth = 7: 0.7295
max_depth = 8: 0.7317
max_depth = 9: 0.7334
max_depth = 10: 0.7354
max_depth = 11: 0.737
max_depth = 12: 0.7379

C-statistic = 0.7523




n = 15:

max_depth = 5: 0.7585
max_depth = 6: 0.7664
max_depth = 7: 0.7742
max_depth = 8: 0.7807
max_depth = 9: 0.7855
max_depth = 10: 0.7895
max_depth = 11: 0.7926
max_depth = 12: 0.7954
max_depth = 13: 0.7974
max_depth = 14: 0.7989
max_depth = 15: 0.8002
max_depth = 16: 0.801

C-statistic = 0.8114




n = 25:

max_depth = 5: 0.7741
max_depth = 6: 0.7818
max_depth = 7: 0.7905
max_depth = 8: 0.7968
max_depth = 9: 0.8014
max_depth = 10: 0.8055
max_depth = 11: 0.8089
max_depth = 12: 0.8114
max_depth = 13: 0.8137
max_depth = 14: 0.8149
max_depth = 15: 0.8162
max_depth = 16: 0.817

C-statistic = 0.8199




n = 35:

max_depth = 5: 0.7775
max_depth = 6: 0.7853
max_depth = 7: 0.7923
max_depth = 8: 0.7986
max_depth = 9: 0.8038
max_depth = 10: 0.8083
max_depth = 11: 0.8118
max_depth = 1

In [22]:
cstat_by_num

[0.7523, 0.8114, 0.8199, 0.8237, 0.8263]